In [14]:
import praw
import pandas as pd
from datetime import datetime, timedelta
import time

# 设置 Reddit API 凭证
reddit = praw.Reddit(
    client_id='Gd_KhZrVZVO79ou3LG_vVw',
    client_secret='-X_P63zpKewLKEreQJWXm2fFXjY1Ew',
    user_agent='reddit-crawler by /u/alpaca_1'
)

# === 参数设置 ===
subreddit_name = "dogecoin"
query = "dogecoin"
target_date = datetime(2025, 4, 23)  # 替换为你想要的日期
next_date = target_date + timedelta(days=1)

# 将目标时间转换为 UTC 时间戳
start_timestamp = int(target_date.timestamp())
end_timestamp = int(next_date.timestamp())

# === 收集帖子 ID ===
subreddit = reddit.subreddit(subreddit_name)
post_ids = []

# 使用 subreddit.search + 时间限制进行搜索
for submission in subreddit.search(query, sort="new", time_filter="all", limit=None):
    if start_timestamp <= submission.created_utc < end_timestamp:
        post_ids.append(submission.id)

print(f"🔍 找到 {len(post_ids)} 篇 {target_date.date()} 当天的包含 'dogecoin' 的帖子")

# === 抓取评论并写入 DataFrame ===
rows = []

for pid in post_ids:
    try:
        submission = reddit.submission(id=pid)
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            rows.append({
                "post_id": pid,
                "author": str(comment.author) if comment.author else "N/A",
                "body": comment.body.replace("\n", " "),
                "score": comment.score,
                "created_utc": comment.created_utc,
                "date": datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
            })
        time.sleep(1)  # 防止被限速
    except Exception as e:
        print(f"⚠️ Error fetching comments for post {pid}: {e}")
        continue

# === 保存为 CSV ===
df = pd.DataFrame(rows, columns=["post_id", "author", "body", "score", "created_utc", "date"])
output_filename = f"dogecoin_comments_{target_date.date()}.csv"
df.to_csv(output_filename, index=False, encoding="utf-8")
print(f"✅ 已保存评论至 {output_filename}，共 {len(df)} 条")


🔍 找到 23 篇 2025-04-23 当天的包含 'dogecoin' 的帖子


/tmp/ipykernel_15362/2738652854.py:48: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "date": datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')


✅ 已保存评论至 dogecoin_comments_2025-04-23.csv，共 318 条
